
- OneHotEncoding can deteriorate the performance of decision trees apparently as it leads to extremely sparse features 

- So let's use this for logistic regression

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
from pprint import pprint
import time
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

In [4]:
acs_income = pd.read_csv('2018_ACSIncome_MA_categories.csv')
acs_income

,AGEP,SCHL,MAR,OCCP,WKHP,SEX,PINCP
0,18.0,SCHL18,Never_Married,EDU,7.0,Female,3000.0
1,28.0,SCHL21,Never_Married,CLN,60.0,Male,35000.0
2,22.0,SCHL19,Never_Married,EDU,25.0,Male,4000.0
3,18.0,SCHL18,Never_Married,SAL,50.0,Male,12000.0
4,20.0,SCHL19,Never_Married,RPR,40.0,Male,5000.0
...,...,...,...,...,...,...,...
40109,21.0,SCHL19,Never_Married,HLS,15.0,Female,15600.0
40110,47.0,SCHL21,Married,CMM,40.0,Female,72000.0
40111,46.0,SCHL21,Married,MGR,50.0,Male,150000.0
40112,36.0,SCHL22,Married,MGR,40.0,Female,78000.0


In [5]:
# no N/A values
acs_income.isna().sum()

AGEP     0
SCHL     0
MAR      0
OCCP     0
WKHP     0
SEX      0
PINCP    0
dtype: int64

In [6]:
acs_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40114 entries, 0 to 40113
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGEP    40114 non-null  float64
 1   SCHL    40114 non-null  object 
 2   MAR     40114 non-null  object 
 3   OCCP    40114 non-null  object 
 4   WKHP    40114 non-null  float64
 5   SEX     40114 non-null  object 
 6   PINCP   40114 non-null  float64
dtypes: float64(3), object(4)
memory usage: 2.1+ MB


In [7]:
acs_income_onehot = acs_income.copy(deep=True)
# drop columns for now and add them back so they're in the same order
acs_income_onehot = acs_income_onehot.drop(acs_income.columns[1:4], axis=1)
acs_income_onehot = acs_income_onehot.drop(acs_income_onehot.columns[2:], axis=1)
acs_income_onehot

,AGEP,WKHP
0,18.0,7.0
1,28.0,60.0
2,22.0,25.0
3,18.0,50.0
4,20.0,40.0
...,...,...
40109,21.0,15.0
40110,47.0,40.0
40111,46.0,50.0
40112,36.0,40.0


# Create one hot encoding

In [8]:
def onehot_enc(col):
    enc = OneHotEncoder(categories='auto',
                        sparse=False,
                        handle_unknown='ignore',
                        dtype='int',
                       )
    
    # shape must be nx1
    original = acs_income[col].values.reshape(-1, 1)
    
    onehot = enc.fit_transform(original)
    print(onehot)
    print(enc.categories_)
    
    # create new features to append
    cols = [cat for cat in enc.categories_[0]]
    df_onehot = pd.DataFrame(onehot, columns=cols)
    
    return df_onehot

## Sex

In [9]:
df_onehot = onehot_enc('SEX')

[[1 0]
 [0 1]
 [0 1]
 ...
 [0 1]
 [1 0]
 [0 1]]
[array(['Female', 'Male'], dtype=object)]


In [10]:
# join the features
acs_income_onehot = acs_income_onehot.join(df_onehot)
acs_income_onehot

,AGEP,WKHP,Female,Male
0,18.0,7.0,1,0
1,28.0,60.0,0,1
2,22.0,25.0,0,1
3,18.0,50.0,0,1
4,20.0,40.0,0,1
...,...,...,...,...
40109,21.0,15.0,1,0
40110,47.0,40.0,1,0
40111,46.0,50.0,0,1
40112,36.0,40.0,1,0


 # SCHL

In [11]:
df_onehot = onehot_enc('SCHL')

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
[array(['SCHL1-15', 'SCHL16', 'SCHL17', 'SCHL18', 'SCHL19', 'SCHL20',
       'SCHL21', 'SCHL22', 'SCHL23', 'SCHL24'], dtype=object)]


In [12]:
# join the features
acs_income_onehot = acs_income_onehot.join(df_onehot)
acs_income_onehot

,AGEP,WKHP,Female,Male,SCHL1-15,SCHL16,SCHL17,SCHL18,SCHL19,SCHL20,SCHL21,SCHL22,SCHL23,SCHL24
0,18.0,7.0,1,0,0,0,0,1,0,0,0,0,0,0
1,28.0,60.0,0,1,0,0,0,0,0,0,1,0,0,0
2,22.0,25.0,0,1,0,0,0,0,1,0,0,0,0,0
3,18.0,50.0,0,1,0,0,0,1,0,0,0,0,0,0
4,20.0,40.0,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40109,21.0,15.0,1,0,0,0,0,0,1,0,0,0,0,0
40110,47.0,40.0,1,0,0,0,0,0,0,0,1,0,0,0
40111,46.0,50.0,0,1,0,0,0,0,0,0,1,0,0,0
40112,36.0,40.0,1,0,0,0,0,0,0,0,0,1,0,0


## MAR

In [13]:
df_onehot = onehot_enc('MAR')

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 ...
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]]
[array(['Divorced', 'Married', 'Never_Married', 'Separated', 'Widowed'],
      dtype=object)]


In [14]:
# join the features
acs_income_onehot = acs_income_onehot.join(df_onehot)
acs_income_onehot

,AGEP,WKHP,Female,Male,SCHL1-15,SCHL16,SCHL17,SCHL18,SCHL19,SCHL20,SCHL21,SCHL22,SCHL23,SCHL24,Divorced,Married,Never_Married,Separated,Widowed
0,18.0,7.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,28.0,60.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,22.0,25.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,18.0,50.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,20.0,40.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40109,21.0,15.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
40110,47.0,40.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
40111,46.0,50.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
40112,36.0,40.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


## OCCP

In [15]:
df_onehot = onehot_enc('OCCP')

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[array(['BUS', 'CLN', 'CMM', 'CMS', 'CON', 'EAT', 'EDU', 'ENG', 'ENT',
       'EXT', 'FFF', 'FIN', 'HLS', 'LGL', 'MED', 'MGR', 'MIL', 'OFF',
       'PRD', 'PRS', 'PRT', 'RPR', 'SAL', 'SCI', 'TRN'], dtype=object)]


In [16]:
# join the features
acs_income_onehot = acs_income_onehot.join(df_onehot)
acs_income_onehot

,AGEP,WKHP,Female,Male,SCHL1-15,SCHL16,SCHL17,SCHL18,SCHL19,SCHL20,...,MGR,MIL,OFF,PRD,PRS,PRT,RPR,SAL,SCI,TRN
0,18.0,7.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28.0,60.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22.0,25.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,18.0,50.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,20.0,40.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40109,21.0,15.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
40110,47.0,40.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40111,46.0,50.0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
40112,36.0,40.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# Add back PINCP

In [17]:
acs_income_onehot['PINCP'] = acs_income['PINCP']
acs_income_onehot

,AGEP,WKHP,Female,Male,SCHL1-15,SCHL16,SCHL17,SCHL18,SCHL19,SCHL20,...,MIL,OFF,PRD,PRS,PRT,RPR,SAL,SCI,TRN,PINCP
0,18.0,7.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3000.0
1,28.0,60.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,35000.0
2,22.0,25.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,4000.0
3,18.0,50.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,12000.0
4,20.0,40.0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40109,21.0,15.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15600.0
40110,47.0,40.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,72000.0
40111,46.0,50.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,150000.0
40112,36.0,40.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,78000.0


In [18]:
acs_income_onehot.to_csv('2018_ACSIncome_MA_onehot.csv', index=False)